In [304]:
# Initial imports
import os
import requests
import urllib3
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from bs4 import BeautifulSoup
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
http = urllib3.PoolManager()

%matplotlib inline

In [305]:
# Load .env enviroment variables
load_dotenv()

True

<h2> Personal Finance Planner

In [306]:
# Set current amount of crypto assets
currency_data = {
    "shares": [1.2, 5.3]
}

# Set the tickers
tickers = ["BTC", "ETH"]

# Create the shares DataFrame
df_currency = pd.DataFrame(currency_data, index=tickers)

# Display shares data
df_currency

,shares
BTC,1.2
ETH,5.3


In [307]:
# Fetch current BTC price
response = http.request('GET', 'https://www.coingecko.com/en/coins/bitcoin')
btc_string = BeautifulSoup(response.data,features="lxml")  
btc_string = str(btc_string)
btc_string = btc_string[btc_string.find('{"@type":"Offer","price":"'):btc_string.find('","priceCurrency":"USD"}}')]
my_btc = float(btc_string[26:])
print(my_btc)


13175.388161344685


In [308]:
# Fetch current ETH price
response = http.request('GET', 'https://www.coingecko.com/en/coins/ethereum')
eth_string = BeautifulSoup(response.data,features="lxml") 
eth_string = str(eth_string)
eth_string = eth_string[eth_string.find('{"@type":"Offer","price":"'):eth_string.find('","priceCurrency":"USD"}}')]
my_eth = float(eth_string[26:])
print(my_eth)



384.65088870170445


In [309]:
# Compute current value of my crpto
my_btc_value = my_btc * df_currency.loc["BTC"]["shares"]
my_eth_value = my_eth * df_currency.loc["ETH"]["shares"]
my_currency_value = (my_btc_value + my_eth_value)

# Print current crypto wallet balance
print(f"The current value of your BTC is ${my_btc_value:0.2f}")
print(f"The current value of your ETH is ${my_eth_value:0.2f}")
print(f"Total currency value: ${my_currency_value:0.2f}")

The current value of your BTC is $15810.47
The current value of your ETH is $2038.65
Total currency value: $17849.12


In [310]:
# Current amount of shares
shares_data = {
    "shares": [200, 50]
}

# Set the tickers
tickers = ["AGG", "SPY"]

# Create the shares DataFrame
df_shares = pd.DataFrame(shares_data, index=tickers)

# Display shares data
df_shares

,shares
AGG,200
SPY,50


In [311]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [312]:
# Format current date as ISO format
today = pd.Timestamp("2020-10-28", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today
).df

# Preview DataFrame
df_portfolio

AGG                                      SPY  \
                             open     high     low   close   volume   open   
2020-10-28 00:00:00-04:00  117.81  117.955  117.72  117.73  5448671  332.1   

                                                                
                               high     low   close     volume  
2020-10-28 00:00:00-04:00  338.2483  326.13  326.67  107939571

In [313]:
# Pick AGG and SPY close prices
my_agg = float(df_portfolio["AGG"]["close"])
my_spy = float(df_portfolio["SPY"]["close"])


# Print AGG and SPY close prices
print(f"Current AGG closing price: ${my_agg}")
print(f"Current SPY closing price: ${my_spy}")


Current AGG closing price: $117.73
Current SPY closing price: $326.67


In [318]:
# Compute the current value of shares
my_agg_value = my_agg * df_shares.loc["AGG"]["shares"]
my_spy_value = my_spy * df_shares.loc["SPY"]["shares"]
my_stock_value = (my_agg_value + my_spy_value)


# Print current value of share
print(f"The current value of the {df_shares.loc['AGG']['shares']} AGG shares is ${my_agg_value:0.2f}")
print(f"The current value of the {df_shares.loc['SPY']['shares']} SPY shares is ${my_spy_value:0.2f}")
print(f"Total stocks: ${my_stock_value:0.2f}")

The current value of the 200 AGG shares is $23546.00
The current value of the 50 SPY shares is $16333.50
Total stocks: $39879.50


In [333]:
# Set monthly household income
monthly_income = 12000

# Create savings DataFrame
total_data = {
    "CRYPTO": [my_currency_value],
    "STOCKS": [my_stock_value]
}

# Create a DataFrame with the current value of shares
df_value = pd.DataFrame(total_data)

# Display DataFrame data
df_value


,CRYPTO,STOCKS
0,17849.115504,39879.5


In [337]:
# Plot savings pie chart
df_value.plot.pie(y="total_data", title="Stocks Portfolio Composition")

KeyError: 'total_data'

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!